In [1]:
PROJECT_ID = "nasa-genie-dev"
REGION = "us-central1"
import os
os.environ["PROJECT_ID"] = PROJECT_ID

In [2]:
import sys
sys.path.append("../../common/src")
sys.path.append("../src")

In [3]:
ONEDRIVE_CLIENT_ID = "821b598e-4662-4f63-a858-e57837256e7f"
os.environ["ONEDRIVE_CLIENT_ID"] = ONEDRIVE_CLIENT_ID
ONEDRIVE_TENANT_ID = "4b9ba20d-5ed9-4ed9-b867-ec77f339bc9c"
os.environ["ONEDRIVE_TENANT_ID"] = ONEDRIVE_TENANT_ID

In [5]:
PG_HOST=None
del os.environ["PG_HOST"]

In [6]:
import os
from typing import List
from common.utils.logging_handler import Logger
from common.models import QueryEngine
from config import (PROJECT_ID,
                    ONEDRIVE_CLIENT_ID,
                    ONEDRIVE_TENANT_ID,
                    ONEDRIVE_CLIENT_SECRET,
                    ONEDRIVE_PRINCIPLE_NAME)
from llama_index.readers.microsoft_onedrive import OneDriveReader
from services.query.data_source import DataSource, DataSourceFile
from services.query.sharepoint_datasource import SharePointDataSource
from utils.gcs_helper import create_bucket, upload_to_gcs

INFO: [config/config.py:57 - <module>()] Namespace File not found, setting job namespace as default


/Users/lramsey/work/ailp/lukmanr-gps-core-solution-services/components/llm_service/notebooks/.venv/lib/python3.9/site-packages/langchain_community/llms/__init__.py:166: LangChainDeprecationWarning: `` was deprecated in LangChain 0.0.22 and will be removed in 0.2. An updated version of the  exists in the langchain-community package and should be used instead. To use it run `pip install -U langchain-community` and import as `from langchain_community.chat_models import ChatDatabricks`.
  warn_deprecated(
/Users/lramsey/work/ailp/lukmanr-gps-core-solution-services/components/llm_service/notebooks/.venv/lib/python3.9/site-packages/langchain_community/llms/__init__.py:329: LangChainDeprecationWarning: `` was deprecated in LangChain 0.0.22 and will be removed in 0.2. An updated version of the  exists in the langchain-community package and should be used instead. To use it run `pip install -U langchain-community` and import as `from langchain_community.chat_models import ChatMlflow`.
  warn_de

ERROR: [config/model_config.py:591 - instantiate_model_class()] Cannot load langchain model class ChatCohere


/Users/lramsey/work/ailp/lukmanr-gps-core-solution-services/components/llm_service/notebooks/.venv/lib/python3.9/site-packages/langchain_community/llms/__init__.py:166: LangChainDeprecationWarning: `` was deprecated in LangChain 0.0.22 and will be removed in 0.2. An updated version of the  exists in the langchain-community package and should be used instead. To use it run `pip install -U langchain-community` and import as `from langchain_community.chat_models import ChatDatabricks`.
  warn_deprecated(
/Users/lramsey/work/ailp/lukmanr-gps-core-solution-services/components/llm_service/notebooks/.venv/lib/python3.9/site-packages/langchain_community/llms/__init__.py:329: LangChainDeprecationWarning: `` was deprecated in LangChain 0.0.22 and will be removed in 0.2. An updated version of the  exists in the langchain-community package and should be used instead. To use it run `pip install -U langchain-community` and import as `from langchain_community.chat_models import ChatMlflow`.
  warn_de

INFO: [config/config.py:107 - <module>()] ENABLE_GOOGLE_LLM = True
INFO: [config/config.py:108 - <module>()] ENABLE_OPENAI_LLM = True
INFO: [config/config.py:109 - <module>()] ENABLE_COHERE_LLM = True
INFO: [config/config.py:110 - <module>()] ENABLE_GOOGLE_MODEL_GARDEN = True
INFO: [config/config.py:111 - <module>()] ENABLE_TRUSS_LLAMA2 = True
INFO: [config/vector_store_config.py:40 - <module>()] Default vector store = [matching_engine]
INFO: [config/vector_store_config.py:49 - <module>()] PG_HOST = [127.0.0.1]
INFO: [config/vector_store_config.py:50 - <module>()] PG_DBNAME = [pgvector]
INFO: [config/onedrive_config.py:30 - <module>()] ONEDRIVE_CLIENT_ID = [821b598e-4662-4f63-a858-e57837256e7f]
INFO: [config/onedrive_config.py:31 - <module>()] ONEDRIVE_TENANT_ID = [4b9ba20d-5ed9-4ed9-b867-ec77f339bc9c]


E0406 12:42:16.893476000 8509799104 ssl_transport_security_utils.cc:105] Corruption detected.
E0406 12:42:16.895114000 8509799104 ssl_transport_security_utils.cc:61] error:100003fc:SSL routines:OPENSSL_internal:SSLV3_ALERT_BAD_RECORD_MAC
E0406 12:42:16.895117000 8509799104 secure_endpoint.cc:302]            Decryption error: TSI_DATA_CORRUPTED


INFO: [utils/text_helper.py:36 - <module>()] loaded spacy model


In [7]:
doc_url = "shpt:///NASA Demo"

In [8]:
from google.cloud import storage
storage_client = storage.Client(project=PROJECT_ID)

In [9]:
from common.models import QueryEngine
q_engine = QueryEngine.find_by_name("Vertex Search")
bucket_name = SharePointDataSource.downloads_bucket_name(q_engine)
shpt_ds = SharePointDataSource(storage_client, bucket_name=bucket_name)

In [10]:
downloaded_docs = []

# extract folder name from url
sharepoint_folder = doc_url.split("shpt://")[1]

if shpt_ds.bucket_name is None:
  Logger.error(
  f"ERROR: Bucket name for SharepointDataSource {doc_url} not set. "
  f"Scraped files not uploaded to Google Cloud Storage")
else:
  # ensure downloads bucket exists, and clear contents
  create_bucket(shpt_ds.storage_client, shpt_ds.bucket_name)

INFO: [utils/gcs_helper.py:47 - create_bucket()] Bucket nasa-genie-dev-downloads-vertex-search already exists.
INFO: [utils/gcs_helper.py:28 - clear_bucket()] Deleting all objects from GCS bucket nasa-genie-dev-downloads-vertex-search
INFO: [utils/gcs_helper.py:35 - clear_bucket()] 0 files deleted


In [11]:
# download files to local storage
loader = OneDriveReader(
    client_id=ONEDRIVE_CLIENT_ID,
    tenant_id=ONEDRIVE_TENANT_ID,
    client_secret=ONEDRIVE_CLIENT_SECRET,
    userprincipalname=ONEDRIVE_PRINCIPLE_NAME,
    folder_path=sharepoint_folder,
    recursive=False
)

In [44]:
access_token = loader._authenticate_with_msal()

In [46]:
temp_dir = '/tmp/nasa'
subfolder_metadata = loader._connect_download_and_return_metadata(
    access_token, 
    temp_dir, 
    sharepoint_folder, 
    False, 
    userprincipalname=ONEDRIVE_PRINCIPLE_NAME,
    isRelativePath=True
)


In [48]:
subfolder_metadata

{'/tmp/nasa/2021_alphabet_annual_report.pdf': {'file_id': '01YUFKCQTIB3XGNELZLZBJUPVPMBPDBWZ6',
  'file_name': '2021_alphabet_annual_report.pdf',
  'created_by_user': 'Lav Jain',
  'created_by_app': None,
  'created_dateTime': '2024-04-04T23:37:23Z',
  'last_modified_by_user': 'Lav Jain',
  'last_modified_by_app': None,
  'last_modified_datetime': '2024-04-04T23:37:23Z'},
 '/tmp/nasa/2022-alphabet-annual-report.pdf': {'file_id': '01YUFKCQQWUSQKEMTSKNCIQT2RLOYV5Q34',
  'file_name': '2022-alphabet-annual-report.pdf',
  'created_by_user': 'Lav Jain',
  'created_by_app': None,
  'created_dateTime': '2024-04-04T23:37:24Z',
  'last_modified_by_user': 'Lav Jain',
  'last_modified_by_app': None,
  'last_modified_datetime': '2024-04-04T23:37:24Z'}}

In [27]:
documents = loader.load_data(recursive=False)

In [28]:
documents

[Document(id_='8e0c6874-e2e4-4fc1-9cc2-714713dedc4d', embedding=None, metadata={'page_label': '1', 'file_name': '2021_alphabet_annual_report.pdf', 'file_id': '01YUFKCQTIB3XGNELZLZBJUPVPMBPDBWZ6', 'created_by_user': 'Lav Jain', 'created_by_app': None, 'created_dateTime': '2024-04-04T23:37:23Z', 'last_modified_by_user': 'Lav Jain', 'last_modified_by_app': None, 'last_modified_datetime': '2024-04-04T23:37:23Z'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='YearinReview2021_Cover.indd   1-3 4/13/22   5:32 PM', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 Document(id_='b127a0e3-2a86-4bfe-8f2d-ff2aac9b3c5b', embedding=None, metadata={'page_label': '2', 'fi

In [30]:
documents[0].metadata

{'page_label': '1',
 'file_name': '2021_alphabet_annual_report.pdf',
 'file_id': '01YUFKCQTIB3XGNELZLZBJUPVPMBPDBWZ6',
 'created_by_user': 'Lav Jain',
 'created_by_app': None,
 'created_dateTime': '2024-04-04T23:37:23Z',
 'last_modified_by_user': 'Lav Jain',
 'last_modified_by_app': None,
 'last_modified_datetime': '2024-04-04T23:37:23Z'}

In [38]:
documents[2].text

'To our investors,\nSundar Pichai\nCEO of Google and AlphabetThe pandemic continues to test the resilience of our \ncommunities and change the ways we work, learn, and ߡonline, greater amounts of information are being created than ever before; it’s an exciting challenge for a company whose mission is to organize the world’s information and make it universally accessible ރ\nIn 2021, our teams approached this mission with new \npurpose and possibility, and continued to develop helpful technologies with a view toward empowering ߡރWe introduced new ways to search visually and ߡބpeople found vaccination sites near them, used Live View to navigate through tricky indoor spaces like ߡރ\nWe also built tools and features that help businesses \nߡon Google Workspace is helping teams work together in a hybrid environment, and our Work Safer program provides Google Cloud customers with the highest ރ\nAs cities and towns began to open up again, we \nߡto train 10 million people and businesses in digit

In [32]:
len(documents)

268

In [35]:
documents[2].text

'To our investors,\nSundar Pichai\nCEO of Google and AlphabetThe pandemic continues to test the resilience of our \ncommunities and change the ways we work, learn, and ߡonline, greater amounts of information are being created than ever before; it’s an exciting challenge for a company whose mission is to organize the world’s information and make it universally accessible ރ\nIn 2021, our teams approached this mission with new \npurpose and possibility, and continued to develop helpful technologies with a view toward empowering ߡރWe introduced new ways to search visually and ߡބpeople found vaccination sites near them, used Live View to navigate through tricky indoor spaces like ߡރ\nWe also built tools and features that help businesses \nߡon Google Workspace is helping teams work together in a hybrid environment, and our Work Safer program provides Google Cloud customers with the highest ރ\nAs cities and towns began to open up again, we \nߡto train 10 million people and businesses in digit